## Imports

In [97]:
# Read stocks
import yfinance as yf

import pandas as pd

# To read external property file
from jproperties import Properties

## Load External Configurations

In [98]:
configs = Properties()

with open('config/yf_stock_picker_p1.properties', 'rb') as config_file:
     configs.load(config_file)

SYMBOLS = configs.get('SYMBOLS').data.split(',') 
TICKER = configs.get('TICKER').data
METRICS = configs.get('METRICS').data.split(',')
WEIGHTS = configs.get('WEIGHTS').data.split(',')

# Convert weights to integers
WEIGHTS = [int(i) for i in WEIGHTS]

## Data Frame to collect metrics

In [99]:
# DF to collect key metrics
key_metrics_df = pd.DataFrame()
# DF to collect exclusions
key_metrics_ex_df = pd.DataFrame()
# Initialize list with empty dictionaries. This collects raw metrics data for each metric
raw_metrics = [{} for sub in range(len(METRICS))]

## Utility method to compute Opearting Margin

In [100]:
# Computes operating margin
# ticker - ticker symbol to calculate operating income
def compute_om(ticker):
    income_stmt_df = ticker.income_stmt
    # Check to see whether Operating Income exists in the income statement first
    if 'Operating Income' in income_stmt_df.index:
        op_income = income_stmt_df.loc['Operating Income'][0]
        tot_revenue = income_stmt_df.loc['Total Revenue'][0]
        op_margin = op_income/tot_revenue
    else:
        # No 'Operating Income' found, use the margin taion from the info section
        op_margin = ticker.info['operatingMargins']
    return op_margin

## Utility method to return the ranking position

In [101]:
# Returns the ranking;  -1 if the given ticker is not found in the metric
# raw_dict - dictionary of values to rank; dictionary -> symbol, value
# order - True for descending order or False for ascending
def get_ranking(raw_dict, order, ticker):
    # Rank the dictionary values in ascending/descending order
    ranked_dict = dict(sorted(raw_dict.items(), key=lambda item: item[1], reverse=order))
    
    if TICKER in ranked_dict:
        return (list(ranked_dict).index(ticker) + 1)
    # No ranking found, due to a metric is not found with the ticker
    return -1

## Utility method to return a key metric

In [102]:
# Returns a key metric row to add it to the DF
# metrict_dict - a metric dictionary -> symbol, value
# metric_name - name of the metric
# order - order for ranking, default is True, i.e., descending
def get_key_metrics_row(metric_dict, metric_name, order=True):
    # Get the ranking
    ranking = get_ranking(metric_dict, order, TICKER)
    if ranking == -1:
        return {'Metric':metric_name, 'Value':'NA', 'Ranking':'NA'}    
    return {'Metric':metric_name, 'Value':metric_dict[TICKER], 'Ranking':f'{(ranking)}/{len(metric_dict)}'}

## Collect Key Metrics data

In [103]:
for symbol in SYMBOLS:
    ticker = yf.Ticker(symbol)
    # Calculate Operating Margin and convert it to percentage
    raw_metrics[0][symbol] = round((compute_om(ticker) * 100), 2)

    # Dividend Yield -> Check to see whether the dividend yield exists for the symbol
    if 'dividendYield' in ticker.info:
        raw_metrics[1][symbol] = round((ticker.info['dividendYield'] * 100), 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[1], 'Reason':'Missing Dividend Yield'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Dividend Cover -> Check to see whether the dividend rate exists for the symbol
    if 'dividendRate' in ticker.info:
        raw_metrics[2][symbol] = round((ticker.info['trailingEps']/ticker.info['dividendRate']), 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[2], 'Reason':'Missing Dividend Rate'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Debt/EBITDA; check it whether total debt exists in income statement
    if 'Total Debt' in ticker.balance_sheet.index:
        debt_to_ebitda = round((ticker.balance_sheet.loc['Total Debt'][0]/ticker.income_stmt.loc['EBITDA'][0]), 2)
        # We only take into ccount with positive ratios or else they will impact rankings
        if debt_to_ebitda >= 0:
            raw_metrics[3][symbol] = debt_to_ebitda
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[3], 'Reason':'Negative Debt/EBITDA ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[3], 'Reason':'Missing Total Debt'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Fwd P/E -> Check to see whether the Fwd P/E exists for the symbol
    if 'forwardPE' in ticker.info:
        fwd_pe = ticker.info['forwardPE']
        if  fwd_pe >= 0:
            raw_metrics[4][symbol] = round(fwd_pe, 2)
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[4], 'Reason':'Negaive Forward P/E ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)        
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[4], 'Reason':'Missing Forward P/E ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # PEG Ration -> Check to see whether the PEG ration exists for the symbol
    if 'pegRatio' in ticker.info:
        peg_ratio = ticker.info['pegRatio']
        if peg_ratio >= 0:
            raw_metrics[5][symbol] = peg_ratio
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[5], 'Reason':'Negaive PEG ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)                    
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[5], 'Reason':'Missing PEG Ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
print('Done')

Done


## Set Key Metrics

In [104]:
for pos in range(len(METRICS)):
    if (pos >= 3):
        # Ranking is on ascending order (lower the better)
        new_row = get_key_metrics_row(raw_metrics[pos], METRICS[pos], False)
    else:
        new_row = get_key_metrics_row(raw_metrics[pos], METRICS[pos])
    key_metrics_df = pd.concat([key_metrics_df, pd.DataFrame([new_row])], ignore_index=True)
key_metrics_df

,Metric,Value,Ranking
0,Operating Margin,-3.57,15/18
1,Dividend Yield,NA,NA
2,Dividend Cover,NA,NA
3,Debt/EBITDA,22.86,15/16
4,P/E ratio,32.49,12/17
5,PEG ratio,1.29,4/16


## Display Key Metrics Exceptions

In [108]:
key_metrics_ex_df.head()

,Symbol,Metric,Reason
0,CDNS,Dividend Yield,Missing Dividend Yield
1,CDNS,Dividend Cover,Missing Dividend Rate
2,WDAY,Dividend Yield,Missing Dividend Yield
3,WDAY,Dividend Cover,Missing Dividend Rate
4,TEAM,Dividend Yield,Missing Dividend Yield


## Calculate the Score for each metric

In [106]:
score = []
for index, row in key_metrics_df.iterrows():
    if row['Ranking'] == 'NA':
        score.append(0.0)
        continue
    rank, total = tuple(map(int, row['Ranking'].split('/')))
    score.append(round(((total - rank) + 1)/total,2))
key_metrics_df['Score'] = score
key_metrics_df

,Metric,Value,Ranking,Score
0,Operating Margin,-3.57,15/18,0.22
1,Dividend Yield,NA,NA,0.00
2,Dividend Cover,NA,NA,0.00
3,Debt/EBITDA,22.86,15/16,0.12
4,P/E ratio,32.49,12/17,0.35
5,PEG ratio,1.29,4/16,0.81


## Calculate the final score

In [107]:
# Normalize the score out of 10
ticker_score = round(((((key_metrics_df['Score'] * WEIGHTS).sum())/sum(WEIGHTS)) * 10), 2)
print(f'Score for {TICKER} is {ticker_score} out of 10')

Score for WDAY is 2.5 out of 10
